# Day 0: Imports and Utility Functions

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import os
import re
import numpy as np
import random
import string
from collections import Counter, defaultdict, namedtuple, deque, OrderedDict
from functools   import lru_cache, reduce
from statistics  import mean, median, mode, stdev, variance
from itertools   import (permutations, combinations, groupby, cycle, 
                         islice, chain, zip_longest, takewhile, dropwhile, count as count_from)
from heapq       import heappush, heappop
from operator    import iand, ior, ilshift, irshift
from numba       import jit

# Day 1: Not Quite Lisp

In [2]:
with open('inputs/day1.txt') as f:
    content = f.read()
print(content.count('(') - content.count(')'))
cnt = 0
for idx, ch in enumerate(content, 1):
    if ch == '(':
        cnt += 1
    else:
        cnt -= 1
        if cnt < 0:
            print(idx)
            break

232
1783


# Day 2: I Was Told There Would Be No Math

In [3]:
paper = 0
ribbon = 0
with open('inputs/day2.txt') as f:
    for line in f:
        a, b, c = tuple(map(int, line.split('x')))
        ribbon += (sum((a, b, c)) - max(a, b, c)) * 2 + a * b * c
        s1, s2, s3 = a * b, a * c, b * c
        paper += min(s1, s2, s3) + sum((s1, s2, s3)) * 2
print(paper)
print(ribbon)

1606483
3842356


# Day 3: Perfectly Spherical Houses in a Vacuum

In [4]:
with open('inputs/day3.txt') as f:
    content = ''.join([line.strip() for line in f])
# print(content)
direction = {'>' : (0, 1), '<' : (0, -1), '^' : (1, 0), 'v' : (-1, 0)}
i, j = 0, 0
visited = set([(i, j)])
for ch in content:
    di, dj = direction[ch]
    i, j = i + di, j + dj
    visited.add((i, j))
print(len(visited))
i, j = 0, 0
visited = set([(i, j)])
for ch in content[::2]:
    di, dj = direction[ch]
    i, j = i + di, j + dj
    visited.add((i, j))
i, j = 0, 0
for ch in content[1::2]:
    di, dj = direction[ch]
    i, j = i + di, j + dj
    visited.add((i, j))
print(len(visited))

2592
2360


# Day 4: The Ideal Stocking Stuffer

In [5]:
import hashlib

def valid_md5(s):
    m = hashlib.md5(s.encode('utf-8'))
    code = m.hexdigest()
    return len(code) >= 5 and all(code[i] == '0' for i in range(5))

s = "iwrupvqb"
# s = "pqrstuv"
print(next(dropwhile(lambda x : not valid_md5(s + str(x)), count_from(0))))

346386


# Day 5: Doesn't He Have Intern-Elves For This?
Old rule
* It contains at least three vowels (aeiou only), like aei, xazegov, or aeiouaeiouaeiou.
* It contains at least one letter that appears twice in a row, like xx, abcdde (dd), or aabbccdd (aa, bb, cc, or dd).
* It does not contain the strings ab, cd, pq, or xy, even if they are part of one of the other requirements.

New rule

* It contains a pair of any two letters that appears at least twice in the string without overlapping, like xyxy (xy) or aabcdefgaa (aa), but not like aaa (aa, but it overlaps).
* It contains at least one letter which repeats with exactly one letter between them, like xyx, abcdefeghi (efe), or even aaa.

In [6]:
def is_nice(s):
    return (sum(ch in 'aeiou' for ch in s) >= 3 
            and any(len(list(g)) >= 2 for key, g in groupby(s)) 
            and all(item not in s for item in ('ab', 'cd', 'pq', 'xy')))
cnt = 0
with open('inputs/day5.txt') as f:
    for line in f:
        cnt += is_nice(line)
print(cnt)

238


In [7]:
def is_nice_new(s):
    d = {}
    flag = False
    for i in range(len(s) - 1):
        x = s[i : i + 2]
        if x in d:
            if i - d[x] > 1:
                flag = True
                break
        else:
            d[x] = i
    return flag and any(s[i] == s[i + 2] for i in range(len(s) - 2))
cnt = 0
with open('inputs/day5.txt') as f:
    for line in f:
        cnt += is_nice_new(line)
print(cnt)

69


# Day 6: Probably a Fire Hazard

In [8]:
lights = np.zeros((1000, 1000))
def parse_coords(lst):
    a, b = lst[0].split(',')
    c, d = lst[1].split(',')
    return tuple(map(int, (a, b, c, d)))

with open('inputs/day6.txt') as f:
    for line in f:
        lst = re.findall('[0-9]+,[0-9]+', line)
        a, b, c, d = parse_coords(lst)
        if line.startswith('turn off'):
            lights[a:c + 1, b:d + 1] = 0
        elif line.startswith('turn on'):
            lights[a:c + 1, b:d + 1] = 1
        else:
            lights[a:c + 1, b:d + 1] = 1 - lights[a:c + 1, b:d + 1]
print(int(np.sum(lights)))

400410


In [9]:
lights = np.zeros((1000, 1000))
def turn_off(x):
    return max(x - 1, 0)
v_turn_off = np.vectorize(turn_off)
with open('inputs/day6.txt') as f:
    for line in f:
        lst = re.findall('[0-9]+,[0-9]+', line)
        a, b, c, d = parse_coords(lst)
        if line.startswith('turn off'):
            lights[a:c + 1, b:d + 1] = v_turn_off(lights[a:c + 1, b:d + 1])
        elif line.startswith('turn on'):
            lights[a:c + 1, b:d + 1] += 1
        else:
            lights[a:c + 1, b:d + 1] += 2
print(int(np.sum(lights)))

15343601


# Day 7: Some Assembly Required

In [10]:
operators = {'OR' : ior, 'AND' : iand, 'NOT' : lambda x : ~x, 'LSHIFT' : ilshift, 'RSHIFT' : irshift}
wires = {}
def digitify(s):
    return int(s) if s.isdigit() else s
lines = [] # lines of dictionary
with open('inputs/day7-1.txt') as f:
    for line in f:
        d = {}
        lh, rh = line.split(' -> ')
        rh = rh.strip('\n')
        rh = digitify(rh)
        d['rh'] = rh
        if line.startswith('NOT'):
            op, lh1 = lh.split(' ')
            lh = [digitify(lh1)]
        elif ' ' not in lh:
            op = None
            lh = [digitify(lh)]
        else:
            lh1, op, lh2 = lh.split(' ')
            lh = [digitify(lh1), digitify(lh2)]
        d['op'] = op
        d['lh'] = lh
        lines.append(d)
def is_valid(d):
    return all(( lh in wires or type(lh) == int) for lh in d['lh'])
nlines = len(lines)
used = [False] * nlines
cnt = 0
while cnt < nlines:
    cnt += 1
    for i, d in enumerate(lines):
        if used[i]: continue
        if is_valid(d):
            break
    used[i] = True
    rh = d['rh']
    lh = d['lh']
    op = d['op']
    if op is None:
        wires[rh] = lh[0] if type(lh[0]) == int else wires[lh[0]]
    elif op == 'NOT':
        wires[rh] = ~lh[0] if type(lh[0]) == int else ~wires[lh[0]]
    else:
        v1, v2 = lh[0] if type(lh[0]) == int else wires[lh[0]], lh[1] if type(lh[1]) == int else wires[lh[1]]
        wires[rh] = operators[op](v1, v2)
print(wires['a'])

14710


# Day 8: Matchsticks

In [11]:
def count_characters(line):
    cnt = 2
    i = 0
    n = len(line)
    while i < n:
        if line[i] == '\\':
            if line[i + 1] in ('"', '\\'):
                i += 2
                cnt += 1
            elif line[i + 1] == 'x':
                i += 4
                cnt += 3
            else:
                i += 1
        else:
            i += 1
    return cnt

with open('inputs/day8.txt', 'r') as f:
    print(sum(count_characters(line.strip()) for line in f))

1333


In [12]:
def count_characters_new(line):
    cnt = 4
    i = 0
    n = len(line)
    while i < n:
        if line[i] == '\\':
            if line[i + 1] in ('"', '\\'):
                i += 2
                cnt += 2
            elif line[i + 1] == 'x':
                i += 4
                cnt += 1
            else:
                i += 1
        else:
            i += 1
    return cnt
with open('inputs/day8.txt', 'r') as f:
    print(sum(count_characters_new(line.strip()) for line in f))

2046


# Day 9: All in a Single Night

In [13]:
graph = defaultdict(dict)
with open('inputs/day9.txt', 'r') as f:
    for line in f:
        a, _, b, _, dis = line.strip().split()
        graph[a][b] = graph[b][a] = int(dis)

In [14]:
def trip(cities):
    return sum(graph[a][b] for a, b in zip(cities, cities[1:]))
print(min(trip(cities) for cities in permutations(graph)))
print(max(trip(cities) for cities in permutations(graph)))

117
909


# Day 10: Elves Look, Elves Say
inputs: 1113222113

In [15]:
s = "1113222113"
for _ in range(50):
    s = ''.join([item for ch, g in groupby(s)
                      for item in (str(len(tuple(g))), ch)])
print(len(s))

3579328


# Day 11: Corporate Policy

In [16]:
# password = "hxbxwxba"
password = "hxbxxyzz"
def isvalid(s):
    return (any(ord(s[i]) + 1 == ord(s[i + 1]) == ord(s[i + 2]) - 1 for i in range(len(s) - 2))
            and all(not ch in s for ch in ('i', 'o', 'l'))
            and sum(ch * 2 in s for ch in string.ascii_lowercase) >= 2
           )
def increment(s):
    carry = 1
    ans = ''
    for i, ch in enumerate(s[::-1]):
        val = ord(ch) - 97
        carry, val = divmod(val + carry, 26)
        ans += chr(val + 97)
        if carry == 0:
            return (ans + s[::-1][i + 1:])[::-1]

while True:
    password = increment(password)
    if isvalid(password):
        print(password)
        break

hxcaabcc


# Day 12: JSAbacusFramework.io

In [17]:
import json
with open('inputs/day12.json') as f:
    data = json.load(f)

In [18]:
# import collections
# def iterate(data):
#     cnt = 0
#     print(data)
#     for key, value in data.items():
#         if isinstance(value, dict):
#             cnt += iterate(value)
#         else:
#             if isinstance(value, collections.Iterable):
#                 cnt += sum([item for item in value if type(item) == int])
#             else:
#                 if type(value) == int:
#                     cnt += value
#     return cnt
# # print(sum(iterate(item) for item in data))
# for item in d

# Day 13: Knights of the Dinner Table 

In [19]:
# Alice would lose 57 happiness units by sitting next to Bob.
graph = defaultdict(dict)
with open('inputs/day13.txt', 'r') as f:
    for line in f:
        a, *rest, b = line.strip().split()
        b = b[:-1]
        lose_or_gain = rest[1]
        v = -int(rest[2]) if lose_or_gain == 'lose' else int(rest[2])
        graph[a][b] = v
# print(graph)

In [20]:
print(max(sum(graph[perm[i]][perm[(i + 1) % len(graph)]] + graph[perm[(i + 1) % len(graph)]][perm[i]]
          for i in range(len(graph)))
          for perm in permutations(graph.keys())))

618


In [21]:
keys = list(graph.keys())
for key in keys:
    graph[key]['YZ'] = 0
    graph['YZ'][key] = 0

In [22]:
len(graph)

9

In [23]:
print(max(sum(graph[perm[i]][perm[(i + 1) % len(graph)]] + graph[perm[(i + 1) % len(graph)]][perm[i]]
          for i in range(len(graph)))
          for perm in permutations(graph.keys())))

601


# Day 14: Reindeer Olympics

In [24]:
def travel(speed, run_time, rest_time, time):
    period = run_time + rest_time
    dis_period = run_time * speed
    q, r = divmod(time, period)
    return q * dis_period + speed * min(run_time, r)

In [25]:
assert(travel(14, 10, 127, 1000) == 1120)
assert(travel(16, 11, 162, 1000) == 1056)

In [26]:
deers = []
with open('inputs/day14.txt', 'r') as f:
    for line in f:
        items = line.strip().split()
        deers.append((int(items[3]), int(items[6]), int(items[13])))
print(deers)

[(8, 8, 53), (13, 4, 49), (20, 7, 132), (12, 4, 43), (9, 5, 38), (10, 4, 37), (3, 37, 76), (9, 12, 97), (37, 1, 36)]


In [27]:
time = 2503
print(max(travel(speed, run_time, rest_time, time) for speed, run_time, rest_time in deers))

2655


In [28]:
awards = [0] * len(deers)
for i in range(1, time + 1):
    distances = [travel(speed, run_time, rest_time, i) 
                 for speed, run_time, rest_time in deers]
    mx = max(distances)
    for j, distance in enumerate(distances):
        if distance == mx:
            awards[j] += 1
print(max(awards))
# print(max(travel(speed, run_time, rest_time, time) + award
#           for award, (speed, run_time, rest_time) in zip(awards, deers)))

1059


# Day 16: Aunt Sue

In [29]:
aunts_info = []
def parse_aunt(line):
    line = line.strip()
    name, rest = line.split(': ', 1)
    idx = int(name[4:])
    ans = {'idx' : idx}
    ans['idx'] = idx
    for item in rest.split(', '):
        left, right = item.split(': ')
        ans[left] = int(right)
    return ans
with open('inputs/day16.txt', 'r') as f:
    for line in f:
        aunts_info.append(parse_aunt(line))

In [30]:
from copy import deepcopy
aunts = deepcopy(aunts_info)
rules = {'children' : 3, 'cats' : 7, 'samoyeds' : 2,
         'pomeranians' : 3, 'akitas' : 0, 'vizslas' : 0, 'goldfish' : 5,
         'trees' : 3, 'cars' : 2, 'perfumes' : 1}
for key, cnt in rules.items():
    aunts = [aunt for aunt in aunts if key not in aunt or aunt[key] == cnt]
assert(len(aunts) == 1)
print(aunts[0]['idx'])

373


In [31]:
aunts = deepcopy(aunts_info)
for key, cnt in rules.items():
    if key in ('cats', 'trees'):
        aunts = [aunt for aunt in aunts if key not in aunt or aunt[key] > cnt]
    elif key in ('pomeranians', 'goldfish'):
        aunts = [aunt for aunt in aunts if key not in aunt or aunt[key] < cnt]
    else:
        aunts = [aunt for aunt in aunts if key not in aunt or aunt[key] == cnt]
assert(len(aunts) == 1)
print(aunts[0]['idx'])

260


# Day 17: No Such Thing as Too Much

In [32]:
nums = []
with open('inputs/day17.txt', 'r') as f:
    for line in f:
        nums.append(int(line))
target = 150
def num_of_combo_limited(nums, target):
    # the number of containers are fixed
    num_of_combo_limited.cnt = 0
    def helper(idx, target):
        if idx == len(nums):
            if target == 0:
                num_of_combo_limited.cnt += 1
            return
        if target < 0:
            return
        helper(idx + 1, target - nums[idx])
        helper(idx + 1, target)
    helper(0, target)
    return num_of_combo_limited.cnt
assert(num_of_combo_limited([20, 15, 5, 10, 5], 25) == 4)
print(num_of_combo_limited(nums, target))

654


In [33]:
def num_of_min_combo(nums, target):
    num_of_min_combo.cnt = 0
    num_of_min_combo.mn = len(nums)
    def helper(num_used, idx, target):
        if idx == len(nums):
            if target == 0:
                if num_used < num_of_min_combo.mn:
                    num_of_min_combo.mn = num_used
                    num_of_min_combo.cnt = 1
                elif num_used == num_of_min_combo.mn:
                    num_of_min_combo.cnt += 1
            return
        if target < 0:
            return
        helper(num_used + 1, idx + 1, target - nums[idx])
        helper(num_used, idx + 1, target)
    helper(0, 0, target)
    return num_of_min_combo.cnt
print(num_of_min_combo(nums, target))

57


# Day 18: Like a GIF For Your Yard

In [34]:
n = 100
grid = np.zeros((n, n), dtype=np.int8)
with open('inputs/day18.txt', 'r') as f:
    for i, line in enumerate(f):
        for j, ch in enumerate(line.strip()):
            if ch == '#':
                grid[i, j] = 1

In [35]:
def update_lights(grid, stuck=False):
    n = grid.shape[0]
    ans = np.zeros((n, n), dtype=np.int8)
    if stuck:
        ans[0, 0] = ans[0, 99] = ans[99, 0] = ans[99, 99] = 1
    for i in range(n):
        for j in range(n):
            cnt = 0
            for di in (-1, 0, 1):
                for dj in (-1, 0, 1):
                    if di == dj == 0:
                        continue
                    if 0 <= i + di < n and 0 <= j + dj < n and grid[i + di][j + dj] == 1:
                        cnt += 1
            if (grid[i][j] and cnt in (2, 3)) or (grid[i][j] == 0 and cnt == 3):
                ans[i][j] = 1
    return ans
# for _ in range(100):
#     grid = update_lights(grid, stuck=False)
# ans = 768
for _ in range(100):
    grid = update_lights(grid, stuck=True)
print(grid.sum())

781


# Day 19: Medicine for Rudolph

In [36]:
rules = []
with open('inputs/day19.txt', 'r') as f:
    lines = [line.strip() for line in f.readlines() if line.strip()]
    for line in lines[:-1]:
        left, right = line.split(' => ')
        rules.append((left, right))
    string = lines[-1]
print(string)
print(rules[0])

CRnCaSiRnBSiRnFArTiBPTiTiBFArPBCaSiThSiRnTiBPBPMgArCaSiRnTiMgArCaSiThCaSiRnFArRnSiRnFArTiTiBFArCaCaSiRnSiThCaCaSiRnMgArFYSiRnFYCaFArSiThCaSiThPBPTiMgArCaPRnSiAlArPBCaCaSiRnFYSiThCaRnFArArCaCaSiRnPBSiRnFArMgYCaCaCaCaSiThCaCaSiAlArCaCaSiRnPBSiAlArBCaCaCaCaSiThCaPBSiThPBPBCaSiRnFYFArSiThCaSiRnFArBCaCaSiRnFYFArSiThCaPBSiThCaSiRnPMgArRnFArPTiBCaPRnFArCaCaCaCaSiRnCaCaSiRnFYFArFArBCaSiThFArThSiThSiRnTiRnPMgArFArCaSiThCaPBCaSiRnBFArCaCaPRnCaCaPMgArSiRnFYFArCaSiThRnPBPMgAr
('Al', 'ThF')


In [37]:
molecules = set()
for left, right in rules:
    n = len(left)
    i = 0
    while i < len(string):
        idx = string.find(left, i)
        if idx == -1:
            break
        else:
            molecules.add(string[:idx] + right + string[idx+n:])
            i = idx + n
print(len(molecules))

509


In [38]:
## BFS does not work
# def molecule_fabrication(target, rules):
#     dq = deque([(target, 0)])
#     seen = set()
#     while dq:
#         s, step = dq.popleft()
#         cands = set()
#         for right, left in rules:
#             n = len(left)
#             i = 0
#             while i < len(s):
#                 idx = s.find(left, i)
#                 if idx == -1:
#                     break
#                 else:
#                     new_s = s[:idx] + right + s[idx + n:]
#                     if new_s == 'e':
#                         return step + 1
#                     if len(new_s) < len(target) and new_s not in seen:
#                         seen.add(new_s)
#                         cands.add((new_s, step + 1))
#                     i = idx + n
#         for cand in cands:
#             dq.append(cand)
# print(molecule_fabrication(string, rules))

# Day 20: Infinite Elves and Infinite Houses

In [39]:
# 29000000
n = 29000000
n //= 10
import math
def count_presents(num):
    return sum((i + num // i if i != num // i else i)
               for i in range(1, int(math.sqrt(num)) + 1) 
               if num % i == 0)
i = 500000
while True:
    if count_presents(i) >= n:
        print(i)
        break
    i += 1

665280


In [40]:
# up to 50
def count_presents_new(num):
    factors = set()
    for i in range(1, int(math.sqrt(num)) + 1):
        if num % i == 0:
            factors.add(i)
            factors.add(num // i)
    return sum(factor for factor in factors if num // factor <= 50)
i = 600000
n = 29000000
while True:
    if count_presents_new(i) * 11 >= n:
        print(i)
        break
    i += 1

705600


# Day 21: RPG Simulator 20XX
Hit Points: 109

Damage: 8

Armor: 2

In [41]:
weapons = [(8, 4, 0), (10, 5, 0), (25, 6, 0), (40, 7, 0), (74, 8, 0)]
armors = [(0, 0, 0), (13, 0, 1), (31, 0, 2), (53, 0, 3), (75, 0, 4), (102, 0, 5)]
rings = [(0, 0, 0), (25, 1, 0), (50, 2, 0), (100, 3, 0), (40, 0, 1), (40, 0, 2), (80, 0, 3)]
def can_win(pp, pd, pa):
    bp, bd, ba = 109, 8, 2
    while True:
        d = max(1, pd - ba)
        bp -= d
        if bp <= 0:
            return True
        d = max(1, bd - pa)
        pp -= d
        if pp <= 0:
            return False
pp = 100
win_costs = []
lose_costs = []
for w in weapons:
    for a in armors:
        for r1 in rings:
            for r2 in rings:
                if r1 == r2 != (0, 0, 0):
                    continue
                cost = w[0] + a[0] + r1[0] + r2[0]
                pd = w[1] + a[1] + r1[1] + r2[1]
                pa = w[2] + a[2] + r1[2] + r2[2]
                if can_win(pp, pd, pa):
                    win_costs.append(cost)
                else:
                    lose_costs.append(cost)
print(min(win_costs))
print(max(lose_costs))

111
188


# Day 22: Wizard Simulator 20XX

# Day 23: Opening the Turing Lock

In [42]:
# hlf r
with open('inputs/day23.txt', 'r') as f:
    program = []
    for line in f:
        ins, *operands = re.findall(r'[\w-]+', line)
        program.append((ins, operands))

def run(registers, program, verbose=False):
    funcs = {'hlf' : lambda x : x // 2,
             'tpl' : lambda x : x * 3,
             'inc' : lambda x : x + 1,
            }
    pc = 0
    n = len(program)
    while 0 <= pc < n:
        ins, operands = program[pc]
        if ins in ('hlf', 'tpl', 'inc'):
            var = operands[0]
            registers[var] = funcs[ins](registers[var])
            pc += 1
        else:
            if ins == 'jmp':
                pc += int(operands[0])
            else:
                var, offset = operands
                if (ins == 'jie' and registers[var] % 2 == 0) or (ins == 'jio' and registers[var] == 1):
                    pc += int(offset)
                else:
                    pc += 1
        if verbose: print(pc, registers)
    return registers['b']

In [43]:
registers = {'a' : 0, 'b' : 0}
print(run(registers, program, verbose=False))

307


In [44]:
registers = {'a' : 1, 'b' : 0}
print(run(registers, program, verbose=False))

160


# Day 24: It Hangs in the Balance

In [45]:
with open('inputs/day24.txt', 'r') as f:
    weights = [int(line) for line in f]

In [46]:
def quantum(weights, groups):
    avg_weight = sum(weights) // groups
    def helper(weights, idx, target, n, cand, ans):
        "populate ans with all candidates with n items and has target sum"
        if len(cand) > n:
            return
        if len(cand) == n:
            if target == 0:
                ans.append(cand)
            return
        for i in range(idx, len(weights)):
            helper(weights, i + 1, target - weights[i], n, cand | {weights[i]}, ans)
    for i in range(1, len(weights) // 2):
        cands = []
        helper(weights, 0, avg_weight, i, set(), cands)
        if cands: # should check if the lefts are separable
            return min(reduce(operator.mul, cand) for cand in cands)
            break

In [47]:
assert(quantum([1, 2, 3, 4, 5, 7, 8, 9, 10, 11], 3) == 99)
print(quantum(weights, groups=3))

10439961859


In [48]:
assert(quantum([1, 2, 3, 4, 5, 7, 8, 9, 10, 11], 4) == 44)
print(quantum(weights, groups=4))

72050269


# Day 25: Let It Snow

Enter the code at row 2978, column 3083.

In [49]:
def gen_code():
    row, col = 1, 1
    cnt = 1
    num = 20151125
    while True:
        yield (row, col, num)
        if row == 1:
            row, col = cnt + 1, 1
            cnt += 1
        else:
            row, col = row - 1, col + 1
        num = (num * 252533) % 33554393

In [50]:
next(num for row, col, num in gen_code() if row == 2978 and col == 3083)

2650453